In [114]:
import pandas as pd
import yfinance as yf
import hvplot.pandas
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [116]:
aark = yf.download('ARKK', '2020-03-01','2022-03-31')

[*********************100%***********************]  1 of 1 completed


In [117]:
aark['Adj Close'].hvplot()

:Curve   [Date]   (Adj Close)

In [100]:
aark

In [101]:
def get_train_test_scaled(df):
    prediction_period = -60
    y,X = df.pop(['Adj Close']), df
    return [MinMaxScaler().fit(train).transform(i) for i in [train,df[prediction_period:]]]

In [102]:
aark_train_scaled, aark_test_scaled = get_train_test_scaled(aark)

In [103]:
aark_model = Sequential()

In [104]:
aark_model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(1, 1)))
aark_model.add(LSTM(50, activation='relu', return_sequences=False))
aark_model.add(Dense(25))
aark_model.add(Dense(1))

In [105]:
aark_model.compile(optimizer='adam', loss='mse')

In [106]:
aark_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 1, 100)            40800     
                                                                 
 lstm_13 (LSTM)              (None, 50)                30200     
                                                                 
 dense_12 (Dense)            (None, 25)                1275      
                                                                 
 dense_13 (Dense)            (None, 1)                 26        
                                                                 
Total params: 72,301
Trainable params: 72,301
Non-trainable params: 0
_________________________________________________________________


In [109]:
aark_model.fit(aark_train_scaled, aark_train_scaled, batch_size=1, epochs=50)

Epoch 1/50
466/466 [==============================] - 2s 1ms/step - loss: 0.0270
Epoch 2/50
466/466 [==============================] - 0s 1ms/step - loss: 7.0937e-04
Epoch 3/50
466/466 [==============================] - 0s 1ms/step - loss: 4.9749e-04
Epoch 4/50
466/466 [==============================] - 0s 1ms/step - loss: 2.3934e-04
Epoch 5/50
466/466 [==============================] - 0s 1ms/step - loss: 4.3317e-05
Epoch 6/50
466/466 [==============================] - 0s 1ms/step - loss: 9.7464e-06
Epoch 7/50
466/466 [==============================] - 0s 1ms/step - loss: 5.1391e-06
Epoch 8/50
466/466 [==============================] - 0s 1ms/step - loss: 3.5960e-06
Epoch 9/50
466/466 [==============================] - 0s 1ms/step - loss: 5.9225e-06
Epoch 10/50
466/466 [==============================] - 0s 1ms/step - loss: 2.4516e-05
Epoch 11/50
466/466 [==============================] - 0s 1ms/step - loss: 1.2219e-05
Epoch 12/50
466/466 [==============================] - 0s 1ms/step 

In [110]:
pd.DataFrame(aark_model.history.history).hvplot()

:Curve   [index]   (loss)